In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os

os.chdir("/content/drive/My Drive/Colab/assignment 2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
myDataFrame2 = pd.read_csv('elderly.csv')
myDataFrame2.head()

,AgreementId,Version,Name,Region,Country,Peace Process,Peace Process Name,Stage,Signed Date,Agreement/Conflict Level,Conflict Nature,Agreement Status,Agreement Text
0,349,1,"Dar-Es-Salaam Declaration on Peace, Security, ...",Africa (excl MENA),Angola/Burundi/Central African Republic/Democr...,6,African Great Lakes: Regional Process,SubPar,2004-11-20,InterIntra,Government,Multiparty signed/agreed,"I. PREAMBLE\r\n\r\n1. We, the Heads of State a..."
1,1437,1,National Action Charter of Bahrain,Middle East and North Africa,Bahrain,10,Bahrain: Reform-based peace process,SubPar,2001-02-15,Intra,Government,Unilateral document,National Action Charter\r\n\r\nPreamble\r\n\r\...
2,1643,1,Agreement between General Smith and General Ml...,Europe and Eurasia,Bosnia and Herzegovina/Yugoslavia (former),125,Bosnia peace process,Pre,1995-07-19,Intra,Government/territory,Multiparty signed/agreed,AGREEMENT BETWEEN GENERAL SMITH AND GENERAL ML...
3,916,1,"Comprehensive Ceasefire Agreement, Pleso",Europe and Eurasia,Bosnia and Herzegovina/Yugoslavia (former),125,Bosnia peace process,Cea,1994-12-23,Intra,Government/territory,Multiparty signed/agreed,Comprehensive Cease-fire Agreement\r\n\r\nThe ...
4,1504,1,Comprehensive Peace Agreement (Carter Agreement),Europe and Eurasia,Bosnia and Herzegovina/Yugoslavia (former),125,Bosnia peace process,Pre,1994-12-21,Intra,Government/territory,Multiparty signed/agreed,Comprehensive Peace Agreement\r\n\r\n \r\n\r\n...


### Agreement Text
*   The last one to describe is the **Agreement Text** within this data. There are many stopwords need to be removed, and keywords need to be extracted.

In [39]:
myDataFrame5=myDataFrame2[['Agreement Text']]
myDataFrame5

,Agreement Text
0,"I. PREAMBLE\r\n\r\n1. We, the Heads of State a..."
1,National Action Charter\r\n\r\nPreamble\r\n\r\...
2,AGREEMENT BETWEEN GENERAL SMITH AND GENERAL ML...
3,Comprehensive Cease-fire Agreement\r\n\r\nThe ...
4,Comprehensive Peace Agreement\r\n\r\n \r\n\r\n...
...,...
81,"Declaration of the 22nd of July, 2014\r\nFor f..."
82,AGREEMENT ON COMPREHENSIVE SOLUTIONS BETWEEN T...
83,Agreement between the Uganda Government and th...
84,National Dialogue Conference Outcomes Document...


Create a new file for future use.

In [40]:
myDataFrame5.to_csv('agreement.csv')

Delete all blank lines in the file and save as another file.

In [41]:
f=open('agreement.csv')
g=open('re_agreement.csv','w')
try:
    while True:
        line=f.readline()
        if len(line)==0:
            break
        if line.count('\n')==len(line):
            continue
        g.write(line)
finally:
    f.close()
    g.close()

Clean the text with spacy method.

In [42]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    ldatokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            ldatokens.append('URL') 
        elif token.orth_.startswith('@'):
            ldatokens.append('SCREEN_NAME')
        else:
            ldatokens.append(token.lower_)
    return ldatokens

Lemmatization with wordnet method.

In [43]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def lemma1(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
from nltk.stem.wordnet import WordNetLemmatizer
def lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Use nltk to remove stopwords.

In [44]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
  def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [lemma1(token) for token in tokens]
    return tokens

# 修改-删shall-失败了

In [34]:
open_stopwords = open('cleanup_words.txt',errors = 'ignore') #Import the stop word list 
stopwords = open_stopwords.readlines()
open_stopwords.close()
stopwords[:8] #Observe the stop word format

['shall\n', 'a\n', 'A\n', 'b\n', 'B\n', 'c\n', 'C\n', 'd\n']

In [22]:
for i in range(len(stopwords)):
    stopwords[i] = stopwords[i].strip() 
stopwords[:8] #Observe the stop word format

['shall', 'a', 'A', 'b', 'B', 'c', 'C', 'd']

# lda

This is the data prepared for lda model, which can also first show some high frenquency words in the text.

In [46]:
import random
text_data = []
with open('re_agreement.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            print(tokens)
            text_data.append(tokens)

['encourage', 'political', 'legal', 'cooperation', 'state', 'region', 'address', 'crime', 'particularly', 'genocide', 'crime', 'humanity', 'crime', 'terrorism']
['sign', 'bertrand', 'presle']
[]
[]
[]
['convoy', 'escort', 'priority', 'military', 'activity']
[]
[]
[]
['private', 'institution', 'open', 'state', 'authorisation', 'within', 'conditions', 'prescribe', 'place', 'state', 'control']
['article']
['title', 'transitory', 'final', 'provisions']
[]
['consider', 'accord', 'final', 'agreement', 'compose', 'contribute', 'fulfilment', 'fundamental', 'right', 'political', 'social', 'economic', 'cultural', 'right', 'right', 'victim', 'conflict', 'truth', 'justice', 'reparation', 'right', 'girl']
[]
[]
['large', 'scale', 'title', 'small', 'medium', 'size', 'rural', 'property', 'legalise', 'protect', 'right', 'pertain', 'small', 'medium', 'size', 'rural', 'property', 'words', 'guarantee', 'right', 'people', 'legitimate', 'owner', 'holder', 'violence', 'never', 'method', 'resolving', 'disput

Use for building matrix.

In [47]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

5 topics for lda model.

In [48]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.035*"shall" + 0.009*"party" + 0.008*"state" + 0.007*"group"')
(1, '0.036*"shall" + 0.013*"commission" + 0.011*"right" + 0.011*"member"')
(2, '0.022*"shall" + 0.012*"right" + 0.010*"national" + 0.010*"service"')
(3, '0.014*"shall" + 0.011*"amnesty" + 0.009*"national" + 0.008*"right"')
(4, '0.013*"person" + 0.012*"state" + 0.011*"government" + 0.009*"speaker"')


3 topics for lda model.

In [49]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.035*"shall" + 0.011*"state" + 0.009*"party" + 0.009*"sudan"')
(1, '0.012*"national" + 0.007*"member" + 0.007*"government" + 0.007*"political"')
(2, '0.028*"shall" + 0.013*"person" + 0.011*"commission" + 0.010*"service"')


10 topics for lda model.

In [50]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.022*"office" + 0.015*"shall" + 0.014*"southern" + 0.013*"sudan"')
(1, '0.013*"person" + 0.013*"meeting" + 0.009*"establish" + 0.009*"jurisdiction"')
(2, '0.033*"service" + 0.020*"party" + 0.015*"amnesty" + 0.013*"minister"')
(3, '0.017*"shall" + 0.012*"public" + 0.011*"property" + 0.010*"humanitarian"')
(4, '0.024*"national" + 0.013*"person" + 0.013*"political" + 0.011*"force"')
(5, '0.022*"force" + 0.016*"member" + 0.015*"national" + 0.013*"shall"')
(6, '0.029*"right" + 0.015*"article" + 0.014*"human" + 0.014*"government"')
(7, '0.048*"shall" + 0.018*"state" + 0.014*"council" + 0.014*"commission"')
(8, '0.062*"shall" + 0.008*"ceasefire" + 0.008*"force" + 0.008*"person"')
(9, '0.014*"government" + 0.011*"council" + 0.011*"person" + 0.011*"multi"')


## Explore

### Use pyldavis to visualize different frequent words under different topics.

In [51]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 6.5MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=907b12df3d432d5dbaf9f3ac0dbad64d0d0855b8bfe874838c503fc66cab9e95
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyldavis


老人 5 主题

In [52]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

老人 3 主题

In [53]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


老人 10 主题

In [54]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
